In [1]:
from uniswap_liquidity import (get_factory, get_token_decimals, abi, 
                               UniV2Router, UniV3Router, UniV3Fee, SushiRouter) #see uniswap_liquidity.py
import os
import logging
import pandas as pd
from datetime import datetime
from web3 import Web3
from multicall import Call, Multicall

# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

token_base = {'WBTC':'0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
              'USDC':'0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
               'DAI':'0x6B175474E89094C44Da98b954EedeAC495271d0F',
               'WETH':'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
             }

token_dict = {'WBTC':'0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
              'USDC':'0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
               'DAI':'0x6B175474E89094C44Da98b954EedeAC495271d0F',
             '1INCH':'0x111111111117dC0aa78b770fA6A738034120C302',
              'AAVE':'0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9',
              'COMP':'0xc00e94Cb662C3520282E6f5717214004A7f26888',
               'DPI':'0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b',
               'FEI':'0x956F47F50A910163D8BF957Cf5846D573E7f87CA',
              'LINK':'0x514910771AF9Ca656af840dff83E8264EcF986CA',
               'SNX':'0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F',
               'UNI':'0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
               'YFI':'0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e',
              'LUNA':'0xd2877702675e6cEb975b4A1dFf9fb7BAF4C91ea9',
               'FTM':'0x4E15361FD6b4BB609Fa63C81A2be19d873717870',
             'SUSHI':'0x6B3595068778DD592e39A122f4f5a5cF09C90fE2',
               'CRV':'0xD533a949740bb3306d119CC777fa900bA034cd52',
               'CVX':'0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B',
               'LDO':'0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32',
             'STETH':'0xDFe66B14D37C77F4E9b180cEb433d1b164f0281D',
              'FRAX':'0x853d955aCEf822Db058eb8505911ED77F175b99e',
              'LUSD':'0x5f98805A4E8be255a32880FDeC7F6728C6568bA0',
              'SUSD':'0x57Ab1ec28D129707052df4dF418D58a2D46d5f51',
              'GUSD':'0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd',
              'USDT':'0xdAC17F958D2ee523a2206206994597C13D831ec7',
              'FXS' :'0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0',
             'SPELL':'0x090185f2135308BaD17527004364eBcC2D37e5F6',
             'LQTY' :'0x6DEA81C8171D0bA574754EF6F8b412F2Ed88c54D',
             'WAVAX':'0x85f138bfEE4ef8e540890CFb48F620571d67Eda3',
               'MIM':'0x99D8a9C45b2ecA8864373A26D1459e3Dff1e17F3',
            }

In [2]:
def main(w3):
    pools = {}

    UniFactory2 = get_factory(w3, abi, UniV2Router)
    logging.info(f'UniFactory2: {UniFactory2}')
    UniFactory3 = get_factory(w3,abi, UniV3Router)
    logging.info(f'UniFactory3: {UniFactory3}')
    SushiFactory = get_factory(w3,abi, SushiRouter)
    logging.info(f'SushiRouter: {SushiRouter}')

    for symbol0 in token_base:
        token0 = Web3.toChecksumAddress(token_base[symbol0])
        base_decimals = get_token_decimals(w3, abi, token0, symbol0)

        logging.info(f'pools count= {len(pools)}..')
        
        logging.debug(f'base = {symbol0}')
        
        for symbol1 in token_dict:
            logging.debug(f'token= {symbol1}')
            token1 = Web3.toChecksumAddress(token_dict[symbol1])
            token_decimals = get_token_decimals(w3, abi, token1, symbol1)

            pool_uni2 = w3.eth.contract(address=UniFactory2, abi=abi).functions.getPair(token0,token1).call()
            pools[pool_uni2] = {'token'        :symbol1,
                                'token_decimals':token_decimals,
                                'base_token'    :symbol0,
                                'base_decimals' :base_decimals,
                                'version'       :'Uniswap V2',
                                'fee' : None,}
            pool_sushi = w3.eth.contract(address=SushiFactory, abi=abi).functions.getPair(token0,token1).call()
            pools[pool_sushi] = {'token'       :symbol1,
                                 'token_decimals':token_decimals,
                                 'base_token'    :symbol0,
                                 'base_decimals' :base_decimals,
                                 'version'       :'Sushiswap',
                                 'fee' : None,}            

            multi_getPool = Multicall([
                                Call(UniFactory3, ['getPool(address,address,uint24)(address)', token0,token1,x], [[x, Web3.toChecksumAddress]]) for x in UniV3Fee
                                ]
                                ,_w3 = w3)

            multi_getPool = multi_getPool()

            pools.update({ multi_getPool[fee]:{'token':symbol1,
                                               'token_decimals':token_decimals,
                                               'base_token':symbol0,
                                               'base_decimals':base_decimals,
                                               'version':'Uniswap V3',
                                               'fee' : fee/10**6}
                          for fee in multi_getPool})

    logging.info(f'pools count= {len(pools)}')

    for pool in list(pools.keys()):
        if pool=='0x0000000000000000000000000000000000000000':
            pools.pop(pool)

    multi_balanceOwn = Multicall([Call(token_dict[pools[x]['token']],
                                      ['balanceOf(address)(uint256)', x],
                                      [[x, None]]
                                     ) for x in pools
                                ]
                                ,_w3 = w3)

    multi_balanceOwn = multi_balanceOwn()


    multi_balanceBase = Multicall([Call(token_base[pools[x]['base_token']],
                                      ['balanceOf(address)(uint256)', x],
                                      [[x, None]]
                                     ) for x in pools
                                ]
                                ,_w3 = w3)

    multi_balanceBase = multi_balanceBase()

    {pools[x].update({  'OwnBalance'           : multi_balanceOwn[x]*10**(-pools[x]['token_decimals']),
                        pools[x]['base_token'] : multi_balanceBase[x]*10**(-pools[x]['base_decimals']),
                     })
         for x in multi_balanceBase}

    logging.info(f'pools final count= {len(pools)}')

    return pools

In [3]:
if __name__ == '__main__':
    logging.basicConfig(
        format='%(asctime)s %(funcName)s %(levelname)s %(message)s',
        level=os.environ.get('LOGLEVEL', 'INFO').upper(),
        datefmt='%Y-%m-%d %H:%M:a%S',
        )
    w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
    logging.info (f'Ethereum connected: {w3_eth.isConnected()}')

    ret  = main(w3_eth)


2022-06-08 11:49:a55 <module> INFO Ethereum connected: True
2022-06-08 11:49:a55 main INFO UniFactory2: 0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f
2022-06-08 11:49:a55 main INFO UniFactory3: 0x1F98431c8aD98523631AE4a59f267346ea31F984
2022-06-08 11:49:a56 main INFO SushiRouter: 0xd9e1cE17f2641f24aE83637ab66a2cca9C378B9F
2022-06-08 11:49:a56 main INFO pools count= 0..
2022-06-08 11:50:a33 main INFO pools count= 35..
2022-06-08 11:51:a04 main INFO pools count= 116..
2022-06-08 11:51:a35 main INFO pools count= 169..
2022-06-08 11:52:a06 main INFO pools count= 286
2022-06-08 11:52:a09 main INFO pools final count= 285


In [4]:
df = pd.DataFrame.from_dict(ret, orient = 'index').reset_index().rename(columns = {'index':'pool'})
df['batchtime'] = datetime.utcnow()
display(df)

,pool,token,token_decimals,base_token,base_decimals,version,fee,OwnBalance,USDC,DAI,WBTC,WETH,batchtime
0,0x004375Dff511095CC5A197A54140a24eFEF3A416,WBTC,8,USDC,6,Uniswap V2,NaN,9.419098e+00,2.917122e+05,NaN,NaN,NaN,2022-06-07 23:52:09.244934
1,0x9a772018FbD77fcD2d25657e5C547BAfF3Fd7D16,WBTC,8,USDC,6,Uniswap V3,0.0005,3.484400e-04,5.016764e+00,NaN,NaN,NaN,2022-06-07 23:52:09.244934
2,0x99ac8cA7087fA4A2A1FB6357269965A2014ABc35,WBTC,8,USDC,6,Uniswap V3,0.0030,2.643433e+03,3.065069e+07,NaN,NaN,NaN,2022-06-07 23:52:09.244934
3,0xCBFB0745b8489973Bf7b334d54fdBd573Df7eF3c,WBTC,8,USDC,6,Uniswap V3,0.0100,9.006113e-01,1.578410e+05,NaN,NaN,NaN,2022-06-07 23:52:09.244934
4,0x231B7589426Ffe1b75405526fC32aC09D44364c4,WBTC,8,DAI,18,Uniswap V2,NaN,6.969189e+00,NaN,217145.652114,NaN,NaN,2022-06-07 23:52:09.244934
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,0xFEbf38B1D34818D4827034f97b7D6D77c79D4997,SPELL,18,WETH,18,Uniswap V3,0.0030,1.355395e+08,NaN,NaN,NaN,51.525802,2022-06-07 23:52:09.244934
281,0x2138a12520C9B45Bb4b2C742923B514D5f662fec,SPELL,18,WETH,18,Uniswap V3,0.0100,7.766000e+02,NaN,NaN,NaN,0.000463,2022-06-07 23:52:09.244934
282,0xB13201b48B1E61593DF055576964D0b3AaB66eA3,LQTY,18,WETH,18,Uniswap V2,NaN,8.675385e+03,NaN,NaN,NaN,5.472269,2022-06-07 23:52:09.244934
283,0xD1D5A4c0eA98971894772Dcd6D2f1dc71083C44E,LQTY,18,WETH,18,Uniswap V3,0.0030,1.951791e+05,NaN,NaN,NaN,337.077828,2022-06-07 23:52:09.244934
